# Scalar interpolation depending on a scalar field

In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import sys, os
sys.path.append(os.path.relpath("..//..//..//src//python"))
import hiped as hp
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
import time

## 1) Mesh and physical field $u$

In [ ]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
#mesh = Mesh(unit_cube.GenerateMesh(maxh=0.2))

fesU =  L2(mesh, order = 0) # works only for lowest order
#fesU = H1(mesh, order = 1) # works only for lowest order

# definition of the scalar physical field
u = GridFunction(fesU)
u.Set(CF(x+y))
Draw(u)

## 2) Interpolation space and variables $x$

In [ ]:
# Interpolation domain
#dom = hp.Domain(6)         # 2D
dom = hp.Domain("prism3")   # 3D

# Definition of the VertexFunctions

f1s = hp.VertexFunction(label = "f1s", f = lambda u : u**2+0.1, dfdu = lambda u : 2*u,    flagNGSolve = True)
f2s = hp.VertexFunction(label = "f2s", f = lambda u : 2*u+3,    dfdu = lambda u : CF(2),  flagNGSolve = True)
f3s = hp.VertexFunction(label = "f3s", f = lambda u : u**3,     dfdu = lambda u : 3*u**2, flagNGSolve = True)
f4s = 2 * f2s / f1s 
f5s = (f2s - f1s)**2
f6s = f1s * 3

# Definition of the penalization(s)
penal = [hp.Penalization("simp", coeffPenal=i+1, reverse=False) for i in range(6)]
#penal = hp.Penalization("simp", coeffPenal=1, reverse=False)

# Definition of the interpolation
labelInterp = "interp"
interp_ngs = hp.Interpolation(domain = dom, children = [f1s,f2s,f3s,f4s,f5s,f6s], label = labelInterp, penalization= penal)

# Initialization of interpolation variable rho

fesRho =  L2(mesh, order = 0) # works only for lowest order
#fesRho = H1(mesh, order = 1) # works only for lowest order

rho_ngs = interp_ngs.setInitialVariable(typeInit = "rand", radius = 2, NGSpace = fesRho) 
rho0 = rho_ngs.copy()
rho_ngs = interp_ngs.projection(rho_ngs)
plt.figure()
interp_ngs.plot(rho_ngs)
plt.show()

## 3) Profiling

In [ ]:
print(f"CPU time (NGSolve) : \n")
t = time.time()
for i in range(100): rho_ngs = interp_ngs.projection(rho_ngs)
t_proj_ngs = time.time()-t
print(f"Time(projection)= {t_proj_ngs *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): w_ngs, dwdx_ngs = interp_ngs.evalBasisFunction(rho_ngs)
t_evalbf_ngs = time.time()-t
print(f"Time(basis functions)= {t_evalbf_ngs *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): f_ngs = interp_ngs.eval(rho_ngs, u, w_ngs) 
t_evalf_ngs = time.time()-t
print(f"Time(eval f)= {t_evalf_ngs *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): dfdu_ngs = interp_ngs.evaldu(rho_ngs, u, w_ngs) 
t_evaldu_ngs = time.time()-t
print(f"Time(eval dfdu)= {t_evaldu_ngs *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): dfdx_ngs = interp_ngs.evaldx(rho_ngs, u, w_ngs, dwdx_ngs) 
t_evaldx_ngs = time.time()-t
print(f"Time(eval dfdx)= {t_evaldx_ngs *1000:.2f} ms (100 runs)")

In [ ]:
# interpolated values
Draw(f_ngs, mesh)

In [ ]:
# dfdu (for Newton Raphson)
Draw(dfdu_ngs, mesh)

In [ ]:
# dfdrho (for optimization)
dim = 0 # derivative along the dimension "dim" of rho
Draw(dfdx_ngs[labelInterp][dim], mesh)

# Comparison with a pure Numpy approach

In [ ]:
f1s = hp.VertexFunction(label = "f1s", f = lambda u : u**2+0.1, dfdu = lambda u : 2*u)
f2s = hp.VertexFunction(label = "f2s", f = lambda u : 2*u+3, dfdu = lambda u : 2*np.ones(u.shape))
f3s = hp.VertexFunction(label = "f3s", f = lambda u : u**3, dfdu = lambda u : 3*u**2)
f4s = 2 * f2s / f1s 
f5s = (f2s - f1s)**2
f6s = f1s *3
interp_np = hp.Interpolation(domain = dom, children = [f1s,f2s,f3s,f4s,f5s,f6s], label = "interp", penalization= penal)
u_np = u.vec.FV().NumPy()[:].reshape(1,1,-1)
rho_np = {'interp' : np.array([gf.vec.FV().NumPy()[:] for gf in rho0['interp']]).T}

In [ ]:
print(f"CPU time (Numpy) : \n")
t = time.time()
for i in range(100): rho_np = interp_np.projection(rho_np)
t_proj_np = time.time()-t
print(f"Time(projection)= {t_proj_np *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): w_np, dwdx_np = interp_np.evalBasisFunction(rho_np)
t_evalbf_np = time.time()-t
print(f"Time(basis functions)= {t_evalbf_np *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): f_np = interp_np.eval(rho_np, u_np, w_np) 
t_evalf_np = time.time()-t
print(f"Time(eval f)= {t_evalf_np *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): dfdu_np = interp_np.evaldu(rho_np, u_np, w_np) 
t_evaldu_np = time.time()-t
print(f"Time(eval dfdu)= {t_evaldu_np *1000:.2f} ms (100 runs)")

t = time.time()
for i in range(100): dfdx_np = interp_np.evaldx(rho_np, u_np, w_np, dwdx_np) 
t_evaldx_np = time.time()-t
print(f"Time(eval dfdx)= {t_evaldx_np *1000:.2f} ms (100 runs)")

In [ ]:
from hiped.utils import gf2array
rho_ngsnp = gf2array(rho_ngs['interp'])[0]
plt.figure()
plt.plot((rho_ngsnp.flatten()-  rho_np['interp'].flatten())/rho_np['interp'].flatten(), label = "projection")
plt.plot((w_ngs['interp'].flatten()-  w_np['interp'].flatten())/w_np['interp'].flatten(), label = "w")
plt.plot((dwdx_ngs['interp'].flatten()-  dwdx_np['interp'].flatten())/dwdx_np['interp'].flatten(), label = "dwdx")
plt.grid(); plt.legend()
plt.xlabel("Component")
plt.ylabel("Relative difference")
plt.show()

In [ ]:
# Interpolation
dimF = 0

f_npngs = GridFunction(fesRho)
f_npngs.vec.FV().NumPy()[:] = f_np[dimF,0,:]
print(f"Relative difference - f[dimF = {dimF}]")
Draw((f_ngs[dimF]-f_npngs)/f_npngs, mesh)

In [ ]:
dimF = 0
dimU = 0

dfdu_npngs = GridFunction(fesRho)
dfdu_npngs.vec.FV().NumPy()[:] = dfdu_np[dimF,dimU,:]
print(f"Relative difference - dfdu[dimF = {dimF}][dimU = {dimU}]")
Draw((dfdu_ngs-dfdu_npngs)/dfdu_ngs, mesh)

In [ ]:
keyX = 'interp'
dimF = 0
dimX = 1
dfdx_npngs = GridFunction(fesRho)
dfdx_npngs.vec.FV().NumPy()[:] = dfdx_np[keyX][dimF,dimX,:]
print(f"Relative difference - dfdx[dimF = {dimF}][dimX = {dimX}]")
Draw((dfdx_ngs[keyX][dimX]-dfdx_npngs)/dfdx_npngs, mesh)